In [38]:
import pandas as pd
import re
import string
import matplotlib.pyplot as plt
import numpy as np
import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [39]:
df = pd.read_csv('data/cannabis.csv')

In [41]:
df = df.dropna()

In [62]:
def predict():
    tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
    dtm = tfidf.fit_transform(df['Description'])
    dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())
    nn = NearestNeighbors(n_neighbors=4, algorithm='kd_tree')
    nn.fit(dtm)

In [63]:
import pickle
# Dump the trained classifier (nn)  with Pickle
pickle_filename = 'model.pkl'
pickled_model = open(pickle_filename, 'wb')  # Open the file to save as pkl file
pickle.dump(nn, pickled_model)
pickled_model.close() # Close the pickle instances

In [64]:
# Loading the saved model
model_pkl = open(pickle_filename, 'rb')
NN_model = pickle.load(model_pkl)
print ("Loaded model :: ", NN_model)  # print to verify

Loaded model ::  NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                 radius=1.0)


In [65]:
import pickle
# Dump the trained classifier (nn)  with Pickle
pickle_filename_1 = 'tfidf.pkl'
pickled_model_1 = open(pickle_filename_1, 'wb')  # Open the file to save as pkl file
pickle.dump(tfidf, pickled_model_1)
pickled_model_1.close() # Close the pickle instances

In [66]:
# Loading the saved model
model_pkl_1 = open(pickle_filename_1, 'rb')
tfidf_model = pickle.load(model_pkl_1)
print ("Loaded model :: ", tfidf_model)  # print to verify

Loaded model ::  TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=5000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)


In [67]:
import json
def recommend(user_input):
    temp_df = NN_model.kneighbors(tfidf_model.transform([user_input]).todense())[1]
    

    #print(temp_df)
    
    for i in range(4):
        info = df.loc[temp_df[0][i]]['Strain']
        info_effects = df.loc[temp_df[0][i]]['Effects']
        info_flavor = df.loc[temp_df[0][i]]['Flavor']
        info_description = df.loc[temp_df[0][i]]['Description']
        info_rating = df.loc[temp_df[0][i]]['Rating']
        
        print(json.dumps(info))
        print(json.dumps(info_effects))
        print(json.dumps(info_flavor))
        print(json.dumps(info_description))
        print(json.dumps(info_rating))
        
        #return json.dumps(info)  #for engineeers, the return does not work in jupyter lab.  Should work in vsCode.
        #return json.dumps(info_effects)
        #return json.dumps(info_flavor)
        #return json.dump(info_description)
        #return json.dumps(info_rating)

In [68]:
recommend('happy,Sleepy,Apricot,Citrus,Grapefruit')

"Pineapple-Thai"
"Relaxed,Creative,Sleepy,Happy,Focused"
"Pineapple,Citrus,Sweet"
"Pineapple Thai is a strain that sets itself apart from the rest with a high 5% CBD content. This flower\u2019s attractiveness comes from its ability to administer powerful pain relief without sedation, embodying true hybrid effects. Although Pineapple Thai is an option for daytime use, its strength may lead to a nighttime preference. Patients who desire the powerful medicinal effects of cannabis without the psychoactive inundation will find Pineapple Thai to be a staple in their arsenal."
4.0
"Pineapple-Trainwreck"
"Relaxed,Euphoric,Uplifted,Happy,Giggly"
"Pineapple,Pine,Woody"
"Bring together the popular clear-headed effects of Pineapple Express and the pain relief qualities of Trainwreck and you get this happy sativa. Pineapple Trainwreck has an earthy aroma and a fruity flavor that usually takes more after its pineapple parent. Long-lasting and potent, this strain hits hard at first, but mellows into 